In [1]:
import tarfile as tr
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.listdir()


['.ipynb_checkpoints',
 '20 news groups classsification.ipynb',
 'aclImdb_v1.tar.gz',
 'HTML AND WEB SCRAPPING EXERCISE.ipynb',
 'IMDB',
 'IMDB REVIEWS.ipynb',
 'IMDB.ipynb',
 'WEB SCRAPPING.ipynb',
 'WORK_FROM_HOME.ipynb']

#### EXTRACTING TAR FILE  INTO FOLDERS

In [ ]:
tr_file=tr.open('aclImdb_v1.tar.gz')
tr_file.extractall('IMDB')
tr_file.close()


#### IMPORTING PACKAGES

In [3]:

from bs4 import BeautifulSoup as bs   # BEAUTIFUL SOUP FOR HTML PARSING

import re  # REGULAR EXPRESSIONS

import string


# NLTK PACKAGES

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


In [4]:
root='C:/Users/anish_yagt7hg/Desktop/internship/Untitled Folder/IMDB/aclImdb/'

root_dir=next(os.walk(root))[1]

root_dir

['test', 'train']

#### PREPARING DATA 

In [5]:
data=[]
labels=[]

for directory in root_dir:
    for i in ['neg','pos']:
        for j in os.listdir(root+directory+'/'+i):
            data.append(os.path.join(root,directory,i,j))
            labels.append(i)
            

        
    
        

In [6]:
#TRAIN DATA
train_data=data[25000:]

In [7]:
def tokenize(data):
    text_lines=[]
    for i in data:
        with open(i,'r',encoding='utf-8') as f:
            text_lines.append(f.read())
            f.close()
        
    return text_lines

In [8]:
def lower_data(string):
    return string.lower()

In [9]:
def strip_html_tags(string):
    soup=bs(string,'html.parser')
    text=soup.text    
    return text

In [10]:
def remove_apostrophe(string):
    text=re.sub(r'[\']','',string)
    return text.lower()

In [11]:
def remove_punctuation(string):
    text=re.sub(r'[^\w\s]',' ',string)
    return text

In [12]:
def remove_stopwords(words):
    stop= stopwords.words('english')
    words=[word for word in words if word not in stop and len(word)>1]
    return words

In [13]:
def tokenize_words(string):
    string=string.strip()
    words=word_tokenize(string)
    return words

In [14]:
def lemmatize(words):
    lmtzr = WordNetLemmatizer()
    words=[lmtzr.lemmatize(word) for word in words]
    return words

In [15]:
def preprocess(string):
    string=lower_data(string)
    string=strip_html_tags(string)
    string=remove_apostrophe(string)
    string=remove_punctuation(string)
    words=tokenize_words(string)
    words=remove_stopwords(words)
    words=lemmatize(words)
     
    
    return words

In [16]:
train_data=tokenize(train_data)  # reading training data

train_labels=labels[25000:]  # labels for training data


train_df = pd.DataFrame(list(zip(train_data,train_labels)),columns=['Review','sentiment'])   # dataframe for training data

In [17]:
train_df.head()

Review sentiment
0  Story of a man who has unnatural feelings for ...       neg
1  Airport '77 starts as a brand new luxury 747 p...       neg
2  This film lacked something I couldn't put my f...       neg
3  Sorry everyone,,, I know this is supposed to b...       neg
4  When I was little my parents took me along to ...       neg

#### APPLYING PREPROCESSING FUNCTION TO REVIEWS

In [19]:
%time train_df['Review']=train_df['Review'].apply(preprocess)

Wall time: 1min 5s


In [20]:
train_df.head()     # AFTER APPLYING PREPROCESSING FUNCTION

Review sentiment
0  [story, man, unnatural, feeling, pig, start, o...       neg
1  [airport, 77, start, brand, new, luxury, 747, ...       neg
2  [film, lacked, something, couldnt, put, finger...       neg
3  [sorry, everyone, know, supposed, art, film, w...       neg
4  [little, parent, took, along, theater, see, in...       neg

In [21]:
X=[" ".join(i) for i in train_df['Review']]             # MAKING TRAINING DATA AFTER PREPROCESSING


In [22]:
len(X)

25000

In [23]:
Y=train_df['sentiment']                       # MAKING LABELS FOR TRAINING DATA AND REHSAPING DATA
Y=Y.values.reshape(-1,1)     

In [24]:
from sklearn.model_selection import train_test_split

In [28]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,random_state=9092,shuffle=True)

In [34]:
print("SHAPE OF X_TRAIN IS:\t{}".format(len(X_train)))

print("\nSHAPE OF X_TEST IS:\t{}".format(len(X_test)))

SHAPE OF X_TRAIN IS:	18750

SHAPE OF X_TEST IS:	6250


#### VECTORIZING THE DATA USING TF-IDF

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
tf_idf=TfidfVectorizer(max_features=7000,strip_accents='ascii',min_df=1,ngram_range=(1,2))

In [39]:
X_train_tf_idf=tf_idf.fit(X_train)     # fitting  TF-IDF 

In [40]:
X_train=X_train_tf_idf.transform(X_train).todense()  # converting sparse matrix into arrays

In [43]:
print(X_train[0])



[[0.        0.        0.0427377 ... 0.        0.        0.       ]]


In [44]:
X_test=X_train_tf_idf.transform(X_test).todense()

#### METRICS

In [45]:
from sklearn.metrics import accuracy_score,recall_score,classification_report

#### MultiNomial Naive Bayes

In [46]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(X_train,Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [47]:
clf_test_pred=clf.predict(X_test)
clf_train_pred=clf.predict(X_train)

In [52]:
print("ACCURACY SCORE OF TRAINING DATA IS :{}".format(accuracy_score(clf_train_pred,Y_train)))
print("\nACCURACY SCORE OF TEST DATA IS : {}".format(accuracy_score(clf_test_pred,Y_test)))
print("\nRecall SCore is :{}".format(recall_score(clf_test_pred,Y_test,pos_label='pos')))

print("\n Classification REport For X_test is  ")

print(classification_report(clf_test_pred,Y_test))


ACCURACY SCORE OF TRAINING DATA IS :0.87888

ACCURACY SCORE OF TEST DATA IS : 0.86784

Recall SCore is :0.8611372918630223

 Classification REport For X_test is  
              precision    recall  f1-score   support

         neg       0.86      0.87      0.87      3067
         pos       0.88      0.86      0.87      3183

   micro avg       0.87      0.87      0.87      6250
   macro avg       0.87      0.87      0.87      6250
weighted avg       0.87      0.87      0.87      6250



#### Gaussian Naive bayes

In [54]:
from sklearn.naive_bayes import GaussianNB

In [55]:
gnb=GaussianNB()
%time gnb.fit(X_train,Y_train)


Wall time: 3.63 s


GaussianNB(priors=None, var_smoothing=1e-09)

In [56]:
gnb_test_pred=gnb.predict(X_test)
gnb_train_pred=gnb.predict(X_train)



In [57]:
print("ACCURACY SCORE OF TRAINING DATA IS :{}".format(accuracy_score(gnb_train_pred,Y_train)))
print("\nACCURACY SCORE OF TEST DATA IS : {}".format(accuracy_score(gnb_test_pred,Y_test)))
print("\nRecall SCore is :{}".format(recall_score(gnb_test_pred,Y_test,pos_label='pos')))

print("\n Classification REport For X_test is  ")

print(classification_report(gnb_test_pred,Y_test))


ACCURACY SCORE OF TRAINING DATA IS :0.8709333333333333

ACCURACY SCORE OF TEST DATA IS : 0.83504

Recall SCore is :0.8432786885245902

 Classification REport For X_test is  
              precision    recall  f1-score   support

         neg       0.85      0.83      0.84      3200
         pos       0.82      0.84      0.83      3050

   micro avg       0.84      0.84      0.84      6250
   macro avg       0.84      0.84      0.84      6250
weighted avg       0.84      0.84      0.84      6250



#### LOGISTIC REGRESSION

In [59]:
from sklearn.linear_model import LogisticRegression


In [61]:
lm_model=LogisticRegression(max_iter=300,solver='sag',n_jobs=-1)

%time lm_model.fit(X_train,Y_train)

Wall time: 39.9 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=300, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False)

In [62]:
lm_model_test_pred = lm_model.predict(X_test)
lm_model_train_pred = lm_model.predict(X_train)

In [63]:
print("ACCURACY SCORE OF TRAINING DATA IS :{}".format(accuracy_score(lm_model_train_pred,Y_train)))
print("\nACCURACY SCORE OF TEST DATA IS : {}".format(accuracy_score(lm_model_test_pred,Y_test)))
print("\nRecall SCore is :{}".format(recall_score(lm_model_test_pred,Y_test,pos_label='pos')))

print("\n Classification REport For X_test is  ")

print(classification_report(lm_model_test_pred,Y_test))


ACCURACY SCORE OF TRAINING DATA IS :0.9237333333333333

ACCURACY SCORE OF TEST DATA IS : 0.88992

Recall SCore is :0.8825745682888541

 Classification REport For X_test is  
              precision    recall  f1-score   support

         neg       0.88      0.90      0.89      3065
         pos       0.90      0.88      0.89      3185

   micro avg       0.89      0.89      0.89      6250
   macro avg       0.89      0.89      0.89      6250
weighted avg       0.89      0.89      0.89      6250



In [66]:
import pickle

In [69]:
with open('logistic_model.pkl','wb') as f:
    pickle.dump(lm_model,f)

#### TEST DATA 

In [76]:
def test(data):
    test_data = data[0:25000]
    test_labels=labels[0:25000]
    test_data=tokenize(test_data)
    test_df= pd.DataFrame(list(zip(test_data,test_labels)),columns=['Review','sentiment'])
    test_df['Review']=test_df['Review'].apply(preprocess)
    
    return test_df

In [77]:
test_df=  test(data)

In [78]:
def test_data_transform (data):
    final_text=[" ".join(i) for i in data]
    
    test =X_train_tf_idf.transform(final_text)   
    
    return test
    

In [73]:
def test_data_predictions(data):
    loaded_model = pickle.load(open('logistic_model.pkl', 'rb'))
    predictions=loaded_model.predict(data)
    return predictions

In [79]:
test_pred = test_data_predictions(test_data_transform(test_df['Review']))

In [80]:
print("\nRecall SCore is :{}".format(recall_score(test_pred,test_df['sentiment'],pos_label='pos')))



Recall SCore is :0.8788745827372437


#### BAG OF WORDS MODEL

In [87]:
X1 = X.copy()
Y1=  Y.copy()

In [89]:
X1[0:5]

['story man unnatural feeling pig start opening scene terrific example absurd comedy formal orchestra audience turned insane violent mob crazy chanting singer unfortunately stay absurd whole time general narrative eventually making putting even era turned cryptic dialogue would make shakespeare seem easy third grader technical level better might think good cinematography future great vilmos zsigmond future star sally kirkland frederic forrest seen briefly',
 'airport 77 start brand new luxury 747 plane loaded valuable painting belonging rich businessman philip stevens james stewart flying bunch vip estate preparation opened public museum also board stevens daughter julie kathleen quinlan son luxury jetliner take planned mid air plane hi jacked co pilot chamber robert foxworth two accomplice banker monte markham wilson michael pataki knock passenger crew sleeping gas plan steal valuable cargo land disused plane strip isolated island making descent chamber almost hit oil rig ocean loses 

In [91]:
from sklearn.feature_extraction.text  import CountVectorizer

In [92]:
bow = CountVectorizer(strip_accents='ascii',ngram_range=(1,2),min_df=1,max_features=7000)

In [93]:
X1_train,X1_test,Y1_train,Y1_test=train_test_split(X1,Y1,shuffle=True,random_state=8989)

In [95]:
print(len(X1_train),len(X1_test))

18750 6250


In [96]:
bow_cv =bow.fit(X1_train)

In [97]:
X1_train = bow.transform(X1_train).toarray()

In [98]:
X1_test = bow.transform(X1_test).toarray()

In [99]:
bow_MNB = MultinomialNB()

bow_MNB.fit(X1_train,Y1_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [101]:
bow_MnB_test_pred = bow_MNB.predict(X1_test)
bow_MnB_train_pred = bow_MNB.predict(X1_train)

#### SCORES FOR VALIDATAION DATA ON MULTINOMIAL NAIVE BAYES

In [103]:
print("ACCURACY SCORE OF TRAINING DATA IS :{}".format(accuracy_score(bow_MnB_train_pred,Y1_train)))
print("\nACCURACY SCORE OF TEST DATA IS : {}".format(accuracy_score(bow_MnB_test_pred,Y1_test)))
print("\nRecall SCore is :{}".format(recall_score(bow_MnB_test_pred,Y1_test,pos_label='pos')))

print("\n Classification REport For X_test is  ")

print(classification_report(bow_MnB_test_pred,Y1_test))


ACCURACY SCORE OF TRAINING DATA IS :0.8684266666666667

ACCURACY SCORE OF TEST DATA IS : 0.85744

Recall SCore is :0.8535645472061657

 Classification REport For X_test is  
              precision    recall  f1-score   support

         neg       0.86      0.86      0.86      3136
         pos       0.86      0.85      0.86      3114

   micro avg       0.86      0.86      0.86      6250
   macro avg       0.86      0.86      0.86      6250
weighted avg       0.86      0.86      0.86      6250



#### GAUSSIAN NAIVE BAYES

In [110]:
bow_gnb = GaussianNB(var_smoothing=0.1)

bow_gnb.fit(X1_train,Y1_train)

GaussianNB(priors=None, var_smoothing=0.1)

In [115]:
bow_gnb_test_pred = bow_gnb.predict(X1_test)
bow_gnb_train_pred = bow_gnb.predict(X1_train)

In [116]:
print("ACCURACY SCORE OF TRAINING DATA IS :{}".format(accuracy_score(bow_gnb_train_pred,Y1_train)))
print("\nACCURACY SCORE OF TEST DATA IS : {}".format(accuracy_score(bow_gnb_test_pred,Y1_test)))
print("\nRecall SCore is :{}".format(recall_score(bow_gnb_test_pred,Y1_test,pos_label='pos')))

print("\n Classification REport For X_test is  ")

print(classification_report(bow_gnb_test_pred,Y1_test))


ACCURACY SCORE OF TRAINING DATA IS :0.5336533333333333

ACCURACY SCORE OF TEST DATA IS : 0.53072

Recall SCore is :0.8636363636363636

 Classification REport For X_test is  
              precision    recall  f1-score   support

         neg       0.99      0.52      0.68      6030
         pos       0.06      0.86      0.11       220

   micro avg       0.53      0.53      0.53      6250
   macro avg       0.53      0.69      0.40      6250
weighted avg       0.96      0.53      0.66      6250



In [114]:
bow_logistic =LogisticRegression(max_iter=300,solver='sag')

bow_logistic.fit(X1_train,Y1_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=300, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

In [117]:
bow_log_test_pred = bow_logistic.predict(X1_test)
bow_log_train_pred = bow_logistic.predict(X1_train)

In [120]:
print("ACCURACY SCORE OF TRAINING DATA IS :{}".format(accuracy_score(bow_log_train_pred,Y1_train)))
print("\nACCURACY SCORE OF TEST DATA IS : {}".format(accuracy_score(bow_log_test_pred,Y1_test)))
print("\nRecall SCore is :{}".format(recall_score(bow_log_test_pred,Y1_test,pos_label='pos')))

print("\n Classification REport For X_test is  ")

print(classification_report(bow_gnb_test_pred,Y1_test))


ACCURACY SCORE OF TRAINING DATA IS :0.9690666666666666

ACCURACY SCORE OF TEST DATA IS : 0.87728

Recall SCore is :0.8687381103360812

 Classification REport For X_test is  
              precision    recall  f1-score   support

         neg       0.99      0.52      0.68      6030
         pos       0.06      0.86      0.11       220

   micro avg       0.53      0.53      0.53      6250
   macro avg       0.53      0.69      0.40      6250
weighted avg       0.96      0.53      0.66      6250

